## Import the libraries

In [23]:
!pip3 install dataframe_image
!pip3 install pycaret
!pip3 install pandas


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
%pip install -U scikit-learn pandas numpy matplotlib "pycaret==3.3.2"


  Using cached scikit_learn-1.7.2-cp310-cp310-win_amd64.whl.metadata (11 kB)
  Using cached pandas-2.3.3-cp310-cp310-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
  Using cached matplotlib-3.10.7-cp310-cp310-win_amd64.whl.metadata (11 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Imports

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.utils import resample
from pycaret.classification import *
#import dataframe_image as dfi
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.over_sampling import SMOTE
import os
from pathlib import Path


In [26]:
!python --version


Python 3.10.11


##### Put the target on the end of the dataframe. Here, as an example, our target is the presence of Long Parameter List.

#### Setup the model creation with pycaret for Long Parameter List detection

In [27]:
df = pd.read_csv('dataset_long_parameter_list.csv')
print(f"Dataset carregado com {df.shape[0]} instâncias e {df.shape[1]-1} features")
print(f"\nDistribuição das classes:")
print(df['smell_label'].value_counts())
print(f"\nPrimeiras 5 linhas:")
df.head()

Dataset carregado com 10696 instâncias e 13 features

Distribuição das classes:
smell_label
non-long-parameter-list    10480
long-parameter-list          216
Name: count, dtype: int64

Primeiras 5 linhas:


,raw_sloc,raw_multi,raw_blank,raw_single_comments,hal_func_N2,hal_func_vocabulary,hal_func_length,hal_func_calculated_length,hal_func_volume,hal_func_difficulty,hal_func_effort,hal_func_time,hal_func_bugs,smell_label
0,9,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
1,2,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
2,3,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
3,3,0,0,0,0,0,0,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,non-long-parameter-list
4,29,0,0,0,13,17,21,54.62919,85.83672,2.708333,232.474449,12.915247,0.028612,non-long-parameter-list


In [28]:
# # Balance dataset using sklearn resampling
# class_counts = df['smell_label'].value_counts()
# print('Class distribution before balancing:')
# print(class_counts)

# max_count = class_counts.max()
# balanced_parts = [
#     resample(group, replace=True, n_samples=max_count, random_state=42)
#     for _, group in df.groupby('smell_label')
# ]

# df = (pd.concat(balanced_parts)
#       .sample(frac=1, random_state=42)
#       .reset_index(drop=True))

# print('\nClass distribution after balancing:')
# print(df['smell_label'].value_counts())
# df.head()


In [29]:
s = setup(
    data=df,
    target='smell_label',
    session_id=42,
    fold=10,
    fix_imbalance=True,
    fix_imbalance_method=SMOTE(k_neighbors=5, random_state=42),  # Aumentar k_neighbors
    feature_selection=True,
    remove_multicollinearity=True,
    multicollinearity_threshold=0.85,  # Reduzir threshold
    normalize=True,
    transformation=True,
    verbose=False
)

[LightGBM] [Info] Number of positive: 7336, number of negative: 7336
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1274
[LightGBM] [Info] Number of data points in the train set: 14672, number of used features: 5
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000


In [30]:
df = get_config('X')
df

print("Shape do dataset processado:", df.shape)
print("\nEstatísticas das features processadas:")
print(df.describe())

# Verificar variância das features
print("\nVariância das features:")
print(df.var().sort_values())

# Verificar correlação
print("\nMatriz de correlação (primeiras 5 features):")
print(df.iloc[:, :5].corr())

# Verificar distribuição das classes depois do processamento
y_train = get_config('y_train')
print("\nDistribuição das classes no treino (após SMOTE):")
print(y_train.value_counts())

Shape do dataset processado: (10696, 13)

Estatísticas das features processadas:
           raw_sloc     raw_multi     raw_blank  raw_single_comments  \
count  10696.000000  10696.000000  10696.000000         10696.000000   
mean       7.923710      2.820213      0.647625             0.039454   
std        9.850741     11.810595      2.759910             0.234745   
min        1.000000      0.000000      0.000000             0.000000   
25%        3.000000      0.000000      0.000000             0.000000   
50%        5.000000      0.000000      0.000000             0.000000   
75%        9.000000      0.000000      0.000000             0.000000   
max      196.000000    263.000000    105.000000             9.000000   

        hal_func_N2  hal_func_vocabulary  hal_func_length  \
count  10696.000000         10696.000000     10696.000000   
mean       5.999813             5.521503         9.353964   
std       20.328536            14.262244        31.975581   
min        0.000000       

# Comparar modelos com configuração otimizada para Long Parameter List
# Usar n_select menor para focar nos melhores modelos

In [31]:
top5_models = compare_models(n_select=13, sort='f1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9292,0.6599,0.9292,0.9622,0.9448,0.0310,0.0370,0.1050
knn,K Neighbors Classifier,0.9159,0.6507,0.9159,0.9654,0.9386,0.0795,0.0997,0.0730
qda,Quadratic Discriminant Analysis,0.7607,0.7336,0.7607,0.9689,0.8456,0.0455,0.0982,0.0510
svm,SVM - Linear Kernel,0.7552,0.7347,0.7552,0.9692,0.8413,0.0456,0.1000,0.0530
nb,Naive Bayes,0.7430,0.7398,0.7430,0.9701,0.8344,0.0507,0.1103,0.0610
ridge,Ridge Classifier,0.7321,0.7412,0.7321,0.9700,0.8273,0.0478,0.1070,0.0660
lda,Linear Discriminant Analysis,0.7321,0.7412,0.7321,0.9700,0.8273,0.0478,0.1070,0.3420
lr,Logistic Regression,0.7242,0.7409,0.7242,0.9699,0.8219,0.0455,0.1036,0.0530
dummy,Dummy Classifier,0.0202,0.5000,0.0202,0.0004,0.0008,0.0000,0.0000,0.0640


In [32]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('LongParameterList_results.latex',index=False)
df3.to_csv('LongParameterList_results.csv',index=False)
#dfi.export(df2, 'Bin_Blob_FS1.png')

In [33]:
from sklearn.metrics import classification_report
from pycaret.classification import predict_model

# Dicionário para armazenar os relatórios por modelo e por classe
reports = {}
# Lista para armazenar os dados
data = []

# Iterar sobre os modelos
for model in top5_models:
    # Gerar predições com o modelo individual
    predictions = predict_model(model)

    # Obter nome do modelo
    model_name = model.__class__.__name__

    # Gerar relatório por classe
    report = classification_report(
        predictions['smell_label'],
        predictions['prediction_label'],
        output_dict=True
    )

    # Armazenar o relatório
    reports[model_name] = report

    # Exibir e armazenar métricas por classe
    print(f"\n📊 Classification Report for {model_name}")
    for smell_class in ['long-parameter-list']:
        if smell_class in report:
            print(f"\nSmell: {smell_class}")
            print(f"  Accuracy: {report['accuracy']:.2f}")
            print(f"  Precision: {report[smell_class]['precision']:.2f}")
            print(f"  Recall: {report[smell_class]['recall']:.2f}")
            print(f"  F1-score: {report[smell_class]['f1-score']:.2f}")

            # Armazenar para eventual DataFrame
            data.append({
                "Model": model_name,
                "Smell": smell_class,
                "Accuracy": report["accuracy"],
                "Precision": report[smell_class]["precision"],
                "Recall": report[smell_class]["recall"],
                "F1-score": report[smell_class]["f1-score"]
            })
    print("=" * 80)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9486,0.6098,0.9486,0.9622,0.9552,0.0433,0.0450



📊 Classification Report for LGBMClassifier

Smell: long-parameter-list
  Accuracy: 0.95
  Precision: 0.05
  Recall: 0.09
  F1-score: 0.07


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9355,0.5630,0.9355,0.9641,0.9491,0.0770,0.0863



📊 Classification Report for KNeighborsClassifier

Smell: long-parameter-list
  Accuracy: 0.94
  Precision: 0.07
  Recall: 0.18
  F1-score: 0.10


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.7787,0.6931,0.7787,0.9661,0.8582,0.0374,0.0739


...


📊 Classification Report for LinearDiscriminantAnalysis

Smell: long-parameter-list
  Accuracy: 0.74
  Precision: 0.04
  Recall: 0.48
  F1-score: 0.07


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.7361,0.6985,0.7361,0.9664,0.8305,0.0318,0.0699



📊 Classification Report for LogisticRegression

Smell: long-parameter-list
  Accuracy: 0.74
  Precision: 0.04
  Recall: 0.48
  F1-score: 0.07


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Dummy Classifier,0.0203,0.5000,0.0203,0.0004,0.0008,0.0000,0.0000



📊 Classification Report for DummyClassifier

Smell: long-parameter-list
  Accuracy: 0.02
  Precision: 0.02
  Recall: 1.00
  F1-score: 0.04


In [34]:
# Criar um DataFrame
df = pd.DataFrame(data)

# Salvar o DataFrame em um arquivo CSV
df.to_csv("longparameterlist_classification_results.csv", index=False)
print(f"Resultados da classificação salvos em 'longparameterlist_classification_results.csv'")
print(f"Total de modelos avaliados: {len(df)}")
print("\nResumo dos resultados:")
print(df[['Model', 'F1-score', 'Precision', 'Recall']].to_string(index=False))

Resultados da classificação salvos em 'longparameterlist_classification_results.csv'
Total de modelos avaliados: 9

Resumo dos resultados:
                        Model  F1-score  Precision   Recall
               LGBMClassifier  0.067797   0.053571 0.092308
         KNeighborsClassifier  0.103896   0.072289 0.184615
QuadraticDiscriminantAnalysis  0.073107   0.039943 0.430769
                SGDClassifier  0.075377   0.041040 0.461538
                   GaussianNB  0.073791   0.040222 0.446154
              RidgeClassifier  0.068660   0.036993 0.476923
   LinearDiscriminantAnalysis  0.068660   0.036993 0.476923
           LogisticRegression  0.068207   0.036730 0.476923
              DummyClassifier  0.039707   0.020256 1.000000


In [35]:
df2=pull()
df3 = pd.DataFrame(df2)
df3.to_latex('longparameterlist_performance.latex',index=False)
df3.to_csv('longparameterlist_performance.csv',index=False)
print("Métricas de desempenho salvas em arquivos .latex e .csv")

Métricas de desempenho salvas em arquivos .latex e .csv


In [36]:
top_models = compare_models(n_select=13, sort='f1')  
best = top_models[0]
best


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9292,0.6599,0.9292,0.9622,0.9448,0.0310,0.0370,0.1130
knn,K Neighbors Classifier,0.9159,0.6507,0.9159,0.9654,0.9386,0.0795,0.0997,0.0780
qda,Quadratic Discriminant Analysis,0.7607,0.7336,0.7607,0.9689,0.8456,0.0455,0.0982,0.0630
svm,SVM - Linear Kernel,0.7552,0.7347,0.7552,0.9692,0.8413,0.0456,0.1000,0.0620
nb,Naive Bayes,0.7430,0.7398,0.7430,0.9701,0.8344,0.0507,0.1103,0.0570
ridge,Ridge Classifier,0.7321,0.7412,0.7321,0.9700,0.8273,0.0478,0.1070,0.0520
lda,Linear Discriminant Analysis,0.7321,0.7412,0.7321,0.9700,0.8273,0.0478,0.1070,0.3560
lr,Logistic Regression,0.7242,0.7409,0.7242,0.9699,0.8219,0.0455,0.1036,0.0680
dummy,Dummy Classifier,0.0202,0.5000,0.0202,0.0004,0.0008,0.0000,0.0000,0.0570


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=42, reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

#### Ensemble the top-5 models

In [37]:
blended_models = blend_models(
  top5_models,
  choose_better=True,
  optimize="f1")

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.8318,0.0000,0.8318,0.9674,0.8901,0.0777,0.1310
1,0.7049,0.0000,0.7049,0.9677,0.8096,0.0312,0.0742
2,0.7196,0.0000,0.7196,0.9680,0.8196,0.0347,0.0800
3,0.6903,0.0000,0.6903,0.9712,0.7991,0.0433,0.1077
4,0.7730,0.0000,0.7730,0.9744,0.8547,0.0806,0.1673
5,0.7316,0.0000,0.7316,0.9701,0.8276,0.0466,0.1052
6,0.7170,0.0000,0.7170,0.9736,0.8175,0.0587,0.1386
7,0.7273,0.0000,0.7273,0.9718,0.8246,0.0539,0.1235
8,0.7981,0.0000,0.7981,0.9695,0.8705,0.0616,0.1174


Original model was better than the blended model, hence it will be returned. NOTE: The display metrics are for the blended model (not the original one).


In [38]:
pred = predict_model(blended_models)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9486,0.6098,0.9486,0.9622,0.9552,0.0433,0.0450


In [39]:
result = pull()

In [40]:
result.to_csv('LongParameterList_ensemble.csv')
print("Resultados do ensemble salvos em 'LongParameterList_ensemble.csv'")

Resultados do ensemble salvos em 'LongParameterList_ensemble.csv'


#### Plot the ensemble model

#### Finalize and save the ensemble model

In [41]:
candidate = best  # ou tuned / best

try:
    final_best = finalize_model(candidate)
except Exception as e:
    print("[Aviso] finalize_model falhou. Reaplicando workaround com fix_imbalance=False...")
    set_config('fix_imbalance', False)   # garante que o PyCaret finalize sem reamostragem
    final_best = finalize_model(candidate)

final_best


Pipeline(memory=Memory(location=None),
         steps=[('label_encoding',
                 TransformerWrapperWithInverse(exclude=None, include=None,
                                               transformer=LabelEncoder())),
                ('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['raw_sloc', 'raw_multi',
                                             'raw_blank', 'raw_single_comments',
                                             'hal_func_N2',
                                             'hal_func_vocabulary',
                                             'hal_func_length',
                                             'hal_func_calculated_length',...
                 LGBMClassifier(boosting_type='gbdt', class_weight=None,
                                colsample_bytree=1.0, importance_type='split',
                                learning_rate=0.1, max_depth=-1,
                                min_child_samples=20, min_child_weight=0.001,
                                min_split_gain=0.0, n_estimators=100, n_jobs=-1,
                                num_leaves=31, objective=None, random_state=42,
                                reg_alpha=0.0, reg_lambda=0.0, subsample=1.0,
                                subsample_for_bin=200000, subsample_freq=0))],
         verbose=False)

In [42]:
save_model(final_best, 'long_parameter_list')
print("Modelo final salvo como 'long_parameter_list.pkl'")

Transformation Pipeline and Model Successfully Saved
Modelo final salvo como 'long_parameter_list.pkl'


#### Salvar os 5 melhores modelos individualmente

In [43]:
# Criar um resumo dos modelos salvos com suas métricas
model_summary = []

for i, model in enumerate(top5_models[:5]):
    model_name = model.__class__.__name__
    
    # Gerar predições para obter métricas
    predictions = predict_model(model)
    
    # Gerar relatório de classificação
    report = classification_report(
        predictions['smell_label'],
        predictions['prediction_label'],
        output_dict=True
    )
    
    # Extrair métricas para long-parameter-list
    if 'long-parameter-list' in report:
        model_summary.append({
            'Model': model_name,
            'Rank': i + 1,
            'F1_Score': report['long-parameter-list']['f1-score'],
            'Precision': report['long-parameter-list']['precision'],
            'Recall': report['long-parameter-list']['recall'],
            'Accuracy': report['accuracy']
        })

# Criar DataFrame com o resumo
summary_df = pd.DataFrame(model_summary)

# Exibir resumo
print("📊 Resumo dos 5 melhores modelos para Long Parameter List:")
print(summary_df.to_string(index=False))

# Salvar resumo em CSV
summary_df.to_csv('long_parameter_list_top5_models_summary.csv', index=False)
print(f"\n📁 Resumo salvo como: long_parameter_list_top5_models_summary.csv")

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.9486,0.6098,0.9486,0.9622,0.9552,0.0433,0.0450


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,K Neighbors Classifier,0.9355,0.5630,0.9355,0.9641,0.9491,0.0770,0.0863


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Quadratic Discriminant Analysis,0.7787,0.6931,0.7787,0.9661,0.8582,0.0374,0.0739


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,SVM - Linear Kernel,0.7706,0,0.7706,0.9667,0.8530,0.0396,0.0801


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Naive Bayes,0.7731,0.6956,0.7731,0.9664,0.8546,0.0380,0.0763


📊 Resumo dos 5 melhores modelos para Long Parameter List:
                        Model  Rank  F1_Score  Precision   Recall  Accuracy
               LGBMClassifier     1  0.067797   0.053571 0.092308  0.948582
         KNeighborsClassifier     2  0.103896   0.072289 0.184615  0.935494
QuadraticDiscriminantAnalysis     3  0.073107   0.039943 0.430769  0.778747
                SGDClassifier     4  0.075377   0.041040 0.461538  0.770645
                   GaussianNB     5  0.073791   0.040222 0.446154  0.773138

📁 Resumo salvo como: long_parameter_list_top5_models_summary.csv


In [44]:
# Salvar os 5 melhores modelos individualmente
print("Salvando os 5 melhores modelos para Long Parameter List detection...")

# Lista para armazenar os modelos finalizados e seus nomes
finalized_models = []

# Iterar sobre os 5 melhores modelos
for i, model in enumerate(top5_models[:5]):
    # Obter nome do modelo
    model_name = model.__class__.__name__
    print(f"\n{i+1}. Processando modelo: {model_name}")
    
    # Finalizar o modelo
    try:
        final_model = finalize_model(model)
        
        # Criar nome descritivo para o arquivo
        filename = f"{model_name.lower()}_long_parameter_list"
        
        # Salvar o modelo
        save_model(final_model, filename)
        print(f"   Modelo salvo como: {filename}.pkl")
        
        # Adicionar à lista de modelos finalizados
        finalized_models.append({
            'name': model_name,
            'filename': filename,
            'model': final_model
        })
        
    except Exception as e:
        print(f"   Erro ao salvar modelo {model_name}: {str(e)}")
        # Tentar com workaround
        try:
            set_config('fix_imbalance', False)
            final_model = finalize_model(model)
            filename = f"long_parameter_list_{model_name.lower()}_f1_fixed"
            save_model(final_model, filename)
            print(f"   Modelo salvo com workaround como: {filename}.pkl")
        except Exception as e2:
            print(f"   Falha ao salvar mesmo com workaround: {str(e2)}")

print(f"\n✅ {len(finalized_models)} modelos salvos com sucesso!")
print("\nModelos disponíveis para Long Parameter List:")
for model_info in finalized_models:
    print(f"  - {model_info['name']}: {model_info['filename']}.pkl")

Salvando os 5 melhores modelos para Long Parameter List detection...

1. Processando modelo: LGBMClassifier
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: lgbmclassifier_long_parameter_list.pkl

2. Processando modelo: KNeighborsClassifier
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: kneighborsclassifier_long_parameter_list.pkl

3. Processando modelo: QuadraticDiscriminantAnalysis
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: quadraticdiscriminantanalysis_long_parameter_list.pkl

4. Processando modelo: SGDClassifier
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: sgdclassifier_long_parameter_list.pkl

5. Processando modelo: GaussianNB
Transformation Pipeline and Model Successfully Saved
   Modelo salvo como: gaussiannb_long_parameter_list.pkl

✅ 5 modelos salvos com sucesso!

Modelos disponíveis para Long Parameter List:
  - LGBMClassifier: lgbmclassifier_long_parameter_list